In [1]:
!python -m pip install --upgrade pip
!pip install -U bitsandbytes pandas transformers peft datasets hf_transfer trl

In [ ]:
import os
import json
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset, Dataset

from make_prompts import generate_prompts
from qlora import load_model_and_tokenizer

with open("config.json", "r") as f:
    cfg = json.load(f)

#### Load Dataset

In [ ]:
dataset = load_dataset(cfg["dataset"], split="train")

In [3]:
# 문자열 최대 길이 제한 해제
pd.set_option("display.max_colwidth", None)

# 컬럼/행 개수도 충분히 넉넉하게
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

df = dataset.to_pandas()
print(df[['query', 'test', 'response']].head(1))

#### baseline 확인

In [ ]:
# Qwen2.5 프롬프트 형식 확인
MODEL_NAME = cfg["dataset"]

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
query = df['query'][0]
response = df['response'][0]

system_msg, user_msg = query.split("### Question:", 1)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg},
    # {"role": "assistant", "content": response}
]

prompt = pipe.tokenizer.apply_chat_template(messages, 
                                            tokenize=False, 
                                            add_generation_prompt=True)


print(prompt)

Device set to use cuda:0


<|im_start|>system
You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests.

<|im_end|>
<|im_start|>user

Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
 
Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = [3,3], target = 6
Output: [0,1]

 
Constraints:

2 <= nums.length <= 104
-109 <= nums[i] <= 109
-109 <= target <= 109
Only one valid answer exists.

 
Follow-up: Can you come up with an algorithm that is less than O(n2) time complexity?

### Format: You will use the following s

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=1.0,
    top_p=0.95,
    add_special_tokens=True
)

print(outputs[0]["generated_text"][len(prompt):])

```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        num_to_index = {}  # Dictionary to store the number and its index
        
        for i, num in enumerate(nums):
            complement = target - num  # Calculate the complement
            
            if complement in num_to_index:
                return [num_to_index[complement], i]  # Return the found indices
            else:
                num_to_index[num] = i  # Store the number and its index
```

This approach uses a dictionary to keep track of the numbers we have seen so far along with their indices. For each number, we calculate the complement (the difference between the target and the current number) and check if this complement has already been seen. If it has, we return the current index and the stored index for the complement. Otherwise, we store the current number and its index in the dictionary. This method achieves linear time complexity, \(O(n)\), which is more effic

#### SFT

In [ ]:
MODEL_NAME = cfg["dataset"]
model, tokenizer = load_model_and_tokenizer(MODEL_NAME)
texts = generate_prompts(df, tokenizer)
train_ds = Dataset.from_dict({"text": texts})
output_dir = "checkpoints"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
sft_config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=cfg["sft"]["num_epochs"],
    per_device_train_batch_size=cfg["sft"]["batch_size"],
    save_strategy="epoch",
    optim=cfg["sft"]["optim"],
    learning_rate=cfg["sft"]["lr"],
    bf16=True,
    completion_only_loss=True,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=sft_config
)

Adding EOS to train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
trainer.save_model(output_dir)

model_path = os.path.join(output_dir, cfg["ckpt_name"])
trainer.save_model(model_path)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.001500
20,0.712900
30,0.569300
40,0.473600
50,0.382300
60,0.327000
70,0.309700
80,0.297700
90,0.272500
100,0.288300


In [23]:
sft_tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
sft_model = AutoPeftModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
sft_model.config.use_cache = True
sft_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [ ]:
test_dataset = load_dataset(cfg["dataset"], split="test")

In [ ]:
test_df = test_dataset.to_pandas()
test_idx = 2

query = test_df['query'][test_idx]
system_msg, question = query.split("### Question:", 1)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": question}
]

prompt = tokenizer.apply_chat_template(messages, 
                                       tokenize=False, 
                                       add_generation_prompt=True)

test_pipeline = pipeline("text-generation", 
                         model=sft_model, 
                         tokenizer=tokenizer, 
                         max_new_tokens=cfg["generation"]["max_new_tokens"])

# 생성
outputs = test_pipeline(
    prompt,
    do_sample=True,
    temperature=cfg["generation"]["temperature"],
    top_p=cfg["generation"]["top_p"],
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Device set to use cuda:0


```python
from typing import List

class Solution:
    def subsequenceCount(self, nums: List[int]) -> int:
        mod = 10**9 + 7
        
        # Sort the numbers to ensure smaller numbers come first, which can help in counting odd sums efficiently
        nums.sort()
        
        # Initialize variables to keep track of the previous even count and total count
        prev_even_count, total_count = 0, 1  # Start with one because we have an empty subsequence by default
        
        # Iterate through each number in the sorted list
        for num in nums:
            new_odd_count = total_count + prev_even_count
            new_even_count = total_count
            
            # Update the counts based on whether the current number is odd or even
            if num & 1:  # Check if the number is odd
                prev_even_count = new_odd_count
                total_count = new_even_count
            else:
                prev_even_count = new_even_count
                tota

In [42]:
print("response:", test_df.iloc[test_idx]["response"])
print("question:", test_df.iloc[test_idx]["query"])

response: To solve this problem, we need to count the number of subsequences in the array `nums` that have an odd sum. Given the constraints, a direct approach that checks all possible subsequences is infeasible due to the exponential number of subsequences. Instead, we can use a more efficient approach based on dynamic programming and combinatorial counting.

### Key Observations:
1. **Odd and Even Elements**: The sum of a subsequence is odd if and only if it contains an odd number of odd elements.
2. **Counting Subsequences**: For each element in the array, we can either include it in a subsequence or not. This gives us \(2^n\) possible subsequences for an array of length \(n\).
3. **Using Combinatorics**: We can count the number of odd elements and even elements separately. The number of ways to choose an odd number of odd elements from the odd elements and any number of even elements is the key to solving this problem.

### Approach:
1. Count the number of odd elements (`odd_count`